## Load relevant libraries

In [190]:
from datetime import date
import pandas as pd
import csv, os
from IPython.display import display
import ipywidgets as widgets


## Functions that need to be run

In [214]:
## template builder if there it is a new game starting that creates an empty file
# STRING -> NONE
# file name -> outputs a file in the file name
def build_template(filename):
    with open(filename, 'w',newline= '') as csvfile:
        rwwriter = csv.writer(csvfile, delimiter = ' ')
        rwwriter.writerow(['Round', 'PandemicNum','Pandemic','City','Added'])
    csvfile.close()

In [215]:
## updates the lists with the things that have happened from a previous file
# LIST, DF -> LIST, LIST, LIST
# list of cities, trial data frame, -> revealed deck, known deck, remains of starting deck
def catchup(city, trial_df):
    # empty decks which will be filled later
    revealed = [];
    known = [];

    for index, row in trial_df.iterrows():
        if row.Pandemic == 1: # if its a pandemics add the 'bottom card' and remove the ones from known and put them into the revealed pile
            revealed.append(row.City)
            city.pop(city.index(row.City))
            known.insert(0,revealed)
            revealed = [];
            
        ## if the known deck and reveal deck are empty pull from the cities list and put them in the revealed deck
        elif not known:
            revealed.append(row.City)
            city.pop(city.index(row.City))
            
        # there are known cards in the known deck pull the the first known list
        elif known:
            revealed.append(row.City)
            known[0].pop(known[0].index(row.City))
            if not known[0]: # remove the first list if its empty
                known.pop(0)
    pandemicNum = trial_df[len(trial_df)].PandemicNum
    roundNum = trial_df[len(trial_df)].Round
    return(revealed, known, city, pandemicNum)
    

In [ ]:
## complete end of games mechanisms
#def EndOfGame():
    

## Start by generating game files 

In [217]:
# determine if its a restarted game or starting an original game
gamestatus = input('Are you starting a new game? Yes=1 No=0:  ')

## where the game_logs are stored
logdir = 'game_logs'
citydir = 'city_logs'

if gamestatus == '0':
    ## get relevant file name information
    filedate = input('What date was your game? MMDDYYYY format:  ')
    month = input('What number month are you on?:')
    retry = input('Is this your second attempt at the month? Yes=1 No=0:  ')
    if retry == '1':
        month += '_2'
    filename = os.path.join(logdir,'PLgame_' + filedate + '_' + month + ".csv")#the saved file name
    ####
elif gamestatus == '1':
    ## find out what month this is and what attempt it is:
    month = input('What number month are you on?:')
    retry = input('Is this your second attempt at the month? Yes=1 No=0:  ')
    if retry == '1':
        month += '_2' 
    ## get the date
    today = date.today()
    filedate = today.strftime("%m%d%y")
    filename = os.path.join(logdir,'PLgame_' + filedate + '_' + month + ".csv")#the saved file name
    build_template(filename)
else:
    print('Oh no! I do not understand that response. Restart please!')
# start game file



Are you starting a new game? Yes=1 No=0:   0
What date was your game? MMDDYYYY format:   022025
What number month are you on?: 3
Is this your second attempt at the month? Yes=1 No=0:   0


## load data files and set up relevant information

In [277]:
## load the trial handler into a pandas structure
trial_df = pd.read_csv(filename, delimiter = ' ')

## gather all the countries to be included 
cities_df = pd.read_csv(os.path.join(citydir,'cities_current.csv'))

## make a list of countries
starting_cities = []
for x,y in zip(cities_df['Cities'], cities_df['Num_of_cards']):
        for z in range(y):
            starting_cities.append(x)

if trial_df.size:
    revealed, known, city, pandemicNum = catchup(starting_cities, trial_df)
else:
    revealed = []
    known = []
    city = starting_cities[:]
    pandemicNum = 0
    roundNum = 0

In [279]:
roundNum = 0
pandemicflip = [2,2,2,3,3,4,4]

for x in range(20):
    # start with added countries
    gameover = int(input('Game over? Yes[1], No[0]:'))
    if gameover:
        trial_df.to_csv(filename, index=False) 
        #EndOfGame
        break
    if roundNum == 0:
        # there are 9 starting cities
        for x in range(6):
            flippedcity = input('New city ' + str(len(revealed)+ 1)) # ask what the country is 
            # update revealed and city lists 
            revealed.append(flippedcity)
            city.pop(city.index(flippedcity))
            pandemichuh = 0 # you can't start on a pandemic
            added = 1
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
    else:        
        # start by asking if the round is an pandemic 
        pandemichuh = int(input('Pandemic? Yes[1], No[0]:'))
        while pandemichuh:
            flippedcity = input('What is the bottom card?:') # determine the bottom card
            # add to the revealed list, remove from the city list, then move the revealed list to the top of the known list
            known.insert(0,revealed)
            revealed.append(newadded) 
            city.pop(city.index(newadded))
            
            revealed = []
            pandemicNum += 1 # increment the number of pandemics
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
            pandemichuh = int(input('Pandemic? Yes[1], No[0]:'))
        # as for each card needed to be flipped this turn
        for cardsRevealed in range(pandemicflip[pandemicNum]):
            flippedcity = input('What city is pulled? '+ str(pandemicflip[pandemicNum]- cardsRevealed) + ' left: ')
            if not known:
                revealed.append(flippedcity)
                city.pop(city.index(flippedcity))
                added = 1
            
            # there are known cards in the known deck pull the the first known list
            elif known:
                revealed.append(flippedcity)
                known[0].pop(known[0].index(flippedcity))
                added = 0
                if not known[0]: # remove the first list if its empty
                    known.pop(0)
            trial_df.loc[len(trial_df)] = [roundNum, pandemicNum,  pandemichuh, flippedcity,  added]
            trial_df.to_csv(filename, index=False) 
    roundNum += 1 # there is more than one row per round so increment appropriately
        
    
    

Game over? Yes[1], No[0]: 1


In [280]:
trial_df

,Round,PandemicNum,Pandemic,City,Added
0,0,0,0,New York,1
1,0,0,0,Sao Paulo,1
2,0,0,0,Tripoli,1
3,0,0,0,London,1
4,0,0,0,Sao Paulo,1
5,0,0,0,New York,1
6,1,0,0,Denver,1
7,1,0,0,New York,1
8,2,1,1,New York,1
9,2,2,1,New York,1


In [ ]:
cities = ['New York','DC','Sao Paulo', 'New York']

clicked = [];
i = 0;

switch = [widgets.Button(description=name) for name in cities]

combined = widgets.HBox([items for items in switch])
out =widgets.Output()

def upon_clicked(btn):
    #print(f'You Just clicked {btn.description}.', end='\x1b\r')
    #for n in range(len(cities)):
    #    switch[n].style.button_color = 'gray'
    btn.style.button_color = 'pink'
    return(btn.description)
    

for n in range(len(cities)):
    switch[n].on_click(upon_clicked)

display(combined)